# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 19 2022 2:46PM,246742,20,ALU0007953,Alumier Labs Inc.,Released
1,Sep 19 2022 2:46PM,246742,20,ALU0007955,Alumier Labs Inc.,Released
2,Sep 19 2022 2:46PM,246742,20,ALUR819267861,Alumier Labs Inc.,Released
3,Sep 19 2022 2:42PM,246741,19,60016181.1,"GUSA Granules USA, Inc.",Released
4,Sep 19 2022 2:24PM,246740,19,ADV80005750,"AdvaGen Pharma, Ltd",Released
5,Sep 19 2022 2:24PM,246740,19,ADV80005755,"AdvaGen Pharma, Ltd",Released
6,Sep 19 2022 2:24PM,246740,19,ADV80005756,"AdvaGen Pharma, Ltd",Released
7,Sep 19 2022 2:24PM,246740,19,ADV80005757,"AdvaGen Pharma, Ltd",Released
8,Sep 19 2022 2:11PM,246739,10,0085993271,ISDIN Corporation,Released
9,Sep 19 2022 1:53PM,246738,10,Enova-10659,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,246738,Released,6
23,246739,Released,1
24,246740,Released,4
25,246741,Released,1
26,246742,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246738,NaN,NaN,6.0
246739,NaN,NaN,1.0
246740,NaN,NaN,4.0
246741,NaN,NaN,1.0
246742,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246677,0.0,1.0,0.0
246693,0.0,1.0,0.0
246700,6.0,11.0,1.0
246701,0.0,0.0,2.0
246708,1.0,7.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246677,0,1,0
246693,0,1,0
246700,6,11,1
246701,0,0,2
246708,1,7,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246677,0,1,0
1,246693,0,1,0
2,246700,6,11,1
3,246701,0,0,2
4,246708,1,7,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246677,,1,
1,246693,,1,
2,246700,6,11,1
3,246701,,,2
4,246708,1,7,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 19 2022 2:46PM,246742,20,Alumier Labs Inc.
3,Sep 19 2022 2:42PM,246741,19,"GUSA Granules USA, Inc."
4,Sep 19 2022 2:24PM,246740,19,"AdvaGen Pharma, Ltd"
8,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation
9,Sep 19 2022 1:53PM,246738,10,Emerginnova
15,Sep 19 2022 1:49PM,246736,10,ISDIN Corporation
37,Sep 19 2022 1:44PM,246734,10,ISDIN Corporation
56,Sep 19 2022 1:44PM,246735,19,"GUSA Granules USA, Inc."
57,Sep 19 2022 1:39PM,246733,10,Yusen – 3D Matrix
58,Sep 19 2022 1:31PM,246732,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 19 2022 2:46PM,246742,20,Alumier Labs Inc.,,,3
1,Sep 19 2022 2:42PM,246741,19,"GUSA Granules USA, Inc.",,,1
2,Sep 19 2022 2:24PM,246740,19,"AdvaGen Pharma, Ltd",,,4
3,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation,,,1
4,Sep 19 2022 1:53PM,246738,10,Emerginnova,,,6
5,Sep 19 2022 1:49PM,246736,10,ISDIN Corporation,,,22
6,Sep 19 2022 1:44PM,246734,10,ISDIN Corporation,,,19
7,Sep 19 2022 1:44PM,246735,19,"GUSA Granules USA, Inc.",,,1
8,Sep 19 2022 1:39PM,246733,10,Yusen – 3D Matrix,,,1
9,Sep 19 2022 1:31PM,246732,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 2:46PM,246742,20,Alumier Labs Inc.,3,,
1,Sep 19 2022 2:42PM,246741,19,"GUSA Granules USA, Inc.",1,,
2,Sep 19 2022 2:24PM,246740,19,"AdvaGen Pharma, Ltd",4,,
3,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation,1,,
4,Sep 19 2022 1:53PM,246738,10,Emerginnova,6,,
5,Sep 19 2022 1:49PM,246736,10,ISDIN Corporation,22,,
6,Sep 19 2022 1:44PM,246734,10,ISDIN Corporation,19,,
7,Sep 19 2022 1:44PM,246735,19,"GUSA Granules USA, Inc.",1,,
8,Sep 19 2022 1:39PM,246733,10,Yusen – 3D Matrix,1,,
9,Sep 19 2022 1:31PM,246732,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 2:46PM,246742,20,Alumier Labs Inc.,3,,
1,Sep 19 2022 2:42PM,246741,19,"GUSA Granules USA, Inc.",1,,
2,Sep 19 2022 2:24PM,246740,19,"AdvaGen Pharma, Ltd",4,,
3,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation,1,,
4,Sep 19 2022 1:53PM,246738,10,Emerginnova,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 2:46PM,246742,20,Alumier Labs Inc.,3.0,NaN,NaN
1,Sep 19 2022 2:42PM,246741,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
2,Sep 19 2022 2:24PM,246740,19,"AdvaGen Pharma, Ltd",4.0,NaN,NaN
3,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation,1.0,NaN,NaN
4,Sep 19 2022 1:53PM,246738,10,Emerginnova,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 19 2022 2:46PM,246742,20,Alumier Labs Inc.,3.0,0.0,0.0
1,Sep 19 2022 2:42PM,246741,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
2,Sep 19 2022 2:24PM,246740,19,"AdvaGen Pharma, Ltd",4.0,0.0,0.0
3,Sep 19 2022 2:11PM,246739,10,ISDIN Corporation,1.0,0.0,0.0
4,Sep 19 2022 1:53PM,246738,10,Emerginnova,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2467250,55.0,18.0,7.0
15,246712,64.0,0.0,0.0
16,986817,1.0,3.0,0.0
19,1480397,8.0,16.0,7.0
20,246742,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2467250,55.0,18.0,7.0
1,15,246712,64.0,0.0,0.0
2,16,986817,1.0,3.0,0.0
3,19,1480397,8.0,16.0,7.0
4,20,246742,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,55.0,18.0,7.0
1,15,64.0,0.0,0.0
2,16,1.0,3.0,0.0
3,19,8.0,16.0,7.0
4,20,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,55.0
1,15,Released,64.0
2,16,Released,1.0
3,19,Released,8.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,7.0,0.0,0.0,7.0,0.0
Executing,18.0,0.0,3.0,16.0,0.0
Released,55.0,64.0,1.0,8.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,7.0,0.0,0.0,7.0,0.0
1,Executing,18.0,0.0,3.0,16.0,0.0
2,Released,55.0,64.0,1.0,8.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,7.0,0.0,0.0,7.0,0.0
1,Executing,18.0,0.0,3.0,16.0,0.0
2,Released,55.0,64.0,1.0,8.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()